In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [3]:
engine_res = pd.read_csv(
    '/Users/brianbrogan/Desktop/KI24/figures/compare_engine_test.csv'
)
print(len(np.unique(engine_res['test_paramID'])))
engine_res.head()

26


,test_paramID,mark,total,n_true_pos,n_false_neg
0,dt_percentile0,2,614,4,6
1,dt_percentile0,3,8104,56,37
2,dt_percentile0,4,7077,28,27
3,dt_percentile0,5,1303,5,3
4,dt_percentile0,6,400,2,4


In [38]:
s1ub_df = engine_res[engine_res['test_paramID']=='dt_percentile0']
#print(s1ub_df['n_true_pos'] / s1ub_df['n_false_neg'])
#np.mean(s1ub_df['n_false_neg'])
tp_fn_ratio = s1ub_df['n_true_pos'] / s1ub_df['n_false_neg'].replace([np.inf, -np.inf], np.nan)
tp_fn_ratio[tp_fn_ratio == np.inf] = 0
print(tp_fn_ratio)

0      0.666667
1      1.513514
2      1.037037
3      1.666667
4      0.500000
         ...   
628    1.500000
629         NaN
630         NaN
631    1.000000
632    0.500000
Length: 633, dtype: float64


In [39]:
comp_df = pd.DataFrame(columns=['test_paramID', '# cells', 'avg. cell area', 'avg. cell tp/fn', 'st. dev. tp/fn'])
for test_paramID in np.unique(engine_res['test_paramID']):
    sub_df = engine_res[engine_res['test_paramID'] == test_paramID]
    zero_den_inds = sub_df[(sub_df['n_true_pos'] != 0) & (sub_df['n_false_neg'] == 0)]
    zero_dub_inds = sub_df[(sub_df['n_true_pos'] == 0) & (sub_df['n_false_neg'] == 0)]
    
    sub_df.loc[zero_den_inds.index, 'n_false_neg'] = 1
    sub_df.loc[zero_dub_inds.index, 'n_false_neg'] = 0
    tp_fn_ratio = sub_df['n_true_pos'] / sub_df['n_false_neg']
    tp_fn_ratio.replace([np.inf, -np.inf], np.nan, inplace=True)
    tp_fn_ratio = tp_fn_ratio.dropna()
    st_dev_tp_fn = round(np.std(tp_fn_ratio), 2) if len(tp_fn_ratio) > 1 else 0

    toAdd = {
        'test_paramID': test_paramID,
        '# cells': sub_df.shape[0],
        'avg. cell area': round(np.mean(sub_df['total'], axis=0), 2), 
        'avg. cell tp/fn': round(np.mean(tp_fn_ratio, axis=0), 2),
        'st. dev. tp/fn': st_dev_tp_fn
    }
    
    comp_df.loc[len(comp_df)] = toAdd

In [40]:
comp_df.head(24)

,test_paramID,# cells,avg. cell area,avg. cell tp/fn,st. dev. tp/fn
0,dilation_kernel_size0,270,3028.44,1.04,0.61
1,dilation_kernel_size1,270,3018.59,1.05,0.64
2,dilation_kernel_size2,279,2927.00,1.07,0.66
3,dilation_kernel_size3,269,3013.23,1.06,0.64
4,dilation_kernel_size4,277,2939.26,1.05,0.66
5,dilation_kernel_size5,269,2988.97,1.06,0.66
6,dt_percentile0,633,1480.19,0.96,0.90
7,dt_percentile1,392,2231.64,1.08,0.93
8,dt_percentile2,269,3013.23,1.06,0.64
9,dt_percentile3,172,4176.91,0.96,0.51


In [41]:
comp_df.to_csv("/Users/brianbrogan/Desktop/KI24/figures/ROC_table.csv", index = False)